In [3]:
import pandas as pd

# Veri setini yükle
csv_path = "/content/drive/My Drive/tweets.csv"
df = pd.read_csv(csv_path)

# Rastgele 50 örnek seç
sample_df = df.sample(n=50, random_state=42).reset_index(drop=True)

# Yeni bir sütun oluştur (manuel duygu tahminleri için)
sample_df['sentiment'] = ""

# Her bir tweeti ekrana yazdır ve kullanıcıdan etiket al
for idx, row in sample_df.iterrows():
    print(f"Tweet {idx+1}/50: {row['text']}")
    print("Duygu etiketini giriniz (Pozitif, Negatif, Nötr):")

    # Geçerli bir etiket girilene kadar döngü devam eder
    while True:
        label = input("Etiket: ").strip().capitalize()
        if label in ["Pozitif", "Negatif", "Nötr"]:
            sample_df.at[idx, 'sentiment'] = label
            break
        else:
            print("Geçersiz etiket! Lütfen 'Pozitif', 'Negatif' veya 'Nötr' giriniz.")

# Etiketlenmiş 50 tweeti yeni bir CSV dosyasına kaydet
output_file = "/content/drive/My Drive/random_50_labeled_tweets.csv"
sample_df.to_csv(output_file, index=False)

print(f"Seçilen 50 tweet etiketlendi ve dosya kaydedildi: {output_file}")


Tweet 1/50: Yandaş kanalları izle. Tek sermayeleri bu zaten. Bu arada model uçak ve kumandalarından alırsan tenis topu falan atarsın seni geliştirir.
Duygu etiketini giriniz (Pozitif, Negatif, Nötr):
Etiket: Negatif
Tweet 2/50: Hak sahibimi kendisi sözleşmesi tapusu var mı?
Duygu etiketini giriniz (Pozitif, Negatif, Nötr):
Etiket: Nötr
Tweet 3/50: O zaman istanbul sözleşmesi kadem mor beyinler savaş açar...
Duygu etiketini giriniz (Pozitif, Negatif, Nötr):
Etiket: Nötr
Tweet 4/50: İstanbul Sözleşmesi Bizimdir!
Vazgeçmiyoruz!

1 Haziran Salı günü istanbul sözleşmesinden vazgeçmediğimizi bildirmek için eylemdeyiz.
17.00'da Aydın Kent Meydanı sessiz eylem
18.00'da Aydın Kent Meydanı Basın açıklaması  
Bütün hak savunucularını 1 Haziranda alana çağırıyoruz!
Duygu etiketini giriniz (Pozitif, Negatif, Nötr):
Etiket: Pozitif
Tweet 5/50: AKPliler istanbul sözleşmesi kaldırılırken bizim 6284 sayılı kanunumuz var sözleşmeye gerek yok demişlerdi. Eğer 6284 kalkacak olursa bu sözleri söyleyen tüm 

In [4]:
!git clone https://github.com/ViralLab/TurkishBERTweet.git

Cloning into 'TurkishBERTweet'...
remote: Enumerating objects: 1259, done.
remote: Counting objects: 100% (759/759), done.
remote: Compressing objects: 100% (469/469), done.
remote: Total 1259 (delta 423), reused 621 (delta 290), pack-reused 500 (from 1)
Receiving objects: 100% (1259/1259), 5.06 MiB | 14.19 MiB/s, done.
Resolving deltas: 100% (648/648), done.


In [6]:
!pip install urlextract

In [7]:
from google.colab import drive
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel, PeftConfig
from sklearn.metrics import accuracy_score
import sys
sys.path.append('/content/TurkishBERTweet')
from Preprocessor import preprocess  # preprocess fonksiyonunu içe aktar

# Google Drive'ı bağla
drive.mount('/content/drive')

# GPU kontrolü ve ayarı
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Kullanılan cihaz:", device)

# Lora adaptör modeli ve yapılandırma
peft_model = "VRLLab/TurkishBERTweet-Lora-SA"
peft_config = PeftConfig.from_pretrained(peft_model)

# Tokenizer yükleme
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path)
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Sınıflar için etiket eşleştirmesi
id2label_sa = {0: "Negatif", 2: "Pozitif", 1: "Nötr"}
label2id_sa = {v: k for k, v in id2label_sa.items()}

# Modeli yükleme ve GPU'ya taşıma
model = AutoModelForSequenceClassification.from_pretrained(
    peft_config.base_model_name_or_path, return_dict=True, num_labels=len(id2label_sa), id2label=id2label_sa
)
model = PeftModel.from_pretrained(model, peft_model)
model.to(device)

# Manuel etiketlenmiş veri setini yükle
csv_path = "/content/drive/MyDrive/random_50_labeled_tweets.csv"  # Manuel etiketlenen veri seti
df = pd.read_csv(csv_path)

# Tweetleri temizle ve NaN değerlerini boş string ile değiştir
df['text'] = df['text'].fillna("")  # NaN değerleri boş string ile değiştir

# Her bir tweeti ön işlemden geçir
df['preprocessed_text'] = df['text'].apply(lambda tweet: preprocess(str(tweet)))

# Tahminler için toplu işlem ayarı
batch_size = 32
predictions = []

# Tahmin işlemi
for i in range(0, len(df), batch_size):
    batch_tweets = df['preprocessed_text'][i:i + batch_size].tolist()
    # Tokenizasyon
    inputs = tokenizer(
        batch_tweets,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Model ile tahmin yap
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        batch_predictions = torch.argmax(logits, dim=-1).cpu().numpy().tolist()
        predictions.extend(batch_predictions)

# Tahminleri sınıf etiketlerine çevir
df['prediction'] = predictions
df['prediction_label'] = df['prediction'].apply(lambda x: id2label_sa[x])

# Modelin doğruluğunu hesapla
df['manual_label_id'] = df['sentiment'].apply(lambda x: label2id_sa.get(x, -1))
accuracy = accuracy_score(df['manual_label_id'], df['prediction'])

print(f"Model doğruluğu: {accuracy:.2%}")

# Sonuçları kaydet
output_file = "/content/drive/My Drive/model_vs_altın_std_comparison.csv"
df.to_csv(output_file, index=False)

print(f"Analiz tamamlandı ve sonuçlar kaydedildi: {output_file}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Kullanılan cihaz: cpu


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/338 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.88M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/652M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at VRLLab/TurkishBERTweet and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.bin:   0%|          | 0.00/3.57M [00:00<?, ?B/s]

Model doğruluğu: 58.00%
Analiz tamamlandı ve sonuçlar kaydedildi: /content/drive/My Drive/model_vs_altın_std_comparison.csv
